<a href="https://colab.research.google.com/github/HigherGround189/EGT309-Team-Harish-Kanna/blob/main/EDA/AI_Solution_Development_Case_Study_Ying_Ray_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup


In [2]:
!gdown --fuzzy https://drive.google.com/file/d/17S8vGBsbaAcuxgwSZLGhOOrrhfaqio7j/view?usp=sharing

Downloading...
From: https://drive.google.com/uc?id=17S8vGBsbaAcuxgwSZLGhOOrrhfaqio7j
To: /content/bmarket.db
100% 3.15M/3.15M [00:00<00:00, 205MB/s]


In [20]:
import sqlite3
import plotly.express as px
import polars as pl

pl.Config(set_tbl_cols=-1, fmt_str_lengths=65535, set_tbl_width_chars=65535)

def lprint(dataframe, num=-1):
    with pl.Config(tbl_rows=num):
        print(dataframe)

In [4]:
conn = sqlite3.connect('/content/bmarket.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_name = cursor.fetchall()[0][0]
print(table_name)

bank_marketing


# Dataset Overview

In [43]:
df_schema = {
    'Client ID': pl.Int64,
    'Age': pl.String,
    'Occupation': pl.Categorical,
    'Marital Status': pl.Categorical,
    'Education Level': pl.Categorical,
    'Credit Default': pl.Categorical,
    'Housing Loan': pl.Categorical,
    'Personal Loan': pl.Categorical,
    'Contact Method': pl.Categorical,
    'Campaign Calls': pl.Int64,
    'Previous Contact Days': pl.Int64,
    'Subscription Status': pl.Categorical
    }

df = pl.read_database(query=f"SELECT * FROM {table_name}", connection=conn, schema_overrides=df_schema)
print(df)

shape: (41_188, 12)
┌───────────┬──────────┬─────────────┬────────────────┬─────────────────────┬────────────────┬──────────────┬───────────────┬────────────────┬────────────────┬───────────────────────┬─────────────────────┐
│ Client ID ┆ Age      ┆ Occupation  ┆ Marital Status ┆ Education Level     ┆ Credit Default ┆ Housing Loan ┆ Personal Loan ┆ Contact Method ┆ Campaign Calls ┆ Previous Contact Days ┆ Subscription Status │
│ ---       ┆ ---      ┆ ---         ┆ ---            ┆ ---                 ┆ ---            ┆ ---          ┆ ---           ┆ ---            ┆ ---            ┆ ---                   ┆ ---                 │
│ i64       ┆ str      ┆ cat         ┆ cat            ┆ cat                 ┆ cat            ┆ cat          ┆ cat           ┆ cat            ┆ i64            ┆ i64                   ┆ cat                 │
╞═══════════╪══════════╪═════════════╪════════════════╪═════════════════════╪════════════════╪══════════════╪═══════════════╪════════════════╪══════════════

In [27]:
print(df.describe())

shape: (9, 13)
┌────────────┬─────────────┬───────────┬────────────┬────────────────┬─────────────────┬────────────────┬──────────────┬───────────────┬────────────────┬────────────────┬───────────────────────┬─────────────────────┐
│ statistic  ┆ Client ID   ┆ Age       ┆ Occupation ┆ Marital Status ┆ Education Level ┆ Credit Default ┆ Housing Loan ┆ Personal Loan ┆ Contact Method ┆ Campaign Calls ┆ Previous Contact Days ┆ Subscription Status │
│ ---        ┆ ---         ┆ ---       ┆ ---        ┆ ---            ┆ ---             ┆ ---            ┆ ---          ┆ ---           ┆ ---            ┆ ---            ┆ ---                   ┆ ---                 │
│ str        ┆ f64         ┆ str       ┆ str        ┆ str            ┆ str             ┆ str            ┆ str          ┆ str           ┆ str            ┆ f64            ┆ f64                   ┆ str                 │
╞════════════╪═════════════╪═══════════╪════════════╪════════════════╪═════════════════╪════════════════╪════════════

In [45]:
def get_unique_values_df(df):
    unique_data = []
    for col in df.columns:
        unique_vals = df[col].unique().to_list()
        unique_data.append({"Column Name": col, "Unique Values": unique_vals})
    unique_df = pl.DataFrame(unique_data)

    with pl.Config(set_fmt_table_cell_list_len=13):
        lprint(unique_df)

    return unique_df

unique_df = get_unique_values_df(df)

shape: (12, 2)
┌───────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ Column Name           ┆ Unique Values                                                                                                                                                   │
│ ---                   ┆ ---                                                                                                                                                             │
│ str                   ┆ list[str]                                                                                                                                                       │
╞═══════════════════════╪═════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╡
│ Client ID             ┆ ["1", "2", "3", "4"

In [61]:
# Get distribution of each column
# Get percentage of each unique value that subscribes

def plot_categorical_countplot(df, col_name):
    # Compute counts and percentages of each value, along with creating the graph's display label
    counts = df[col_name].value_counts()
    counts = counts.with_columns(
        (pl.col("count") / df.shape[0] * 100).round(2).alias("percentage")
    ).with_columns(
        # Outputs in this format: <COUNT> (<PERCENT>%)
        (pl.col("count").cast(pl.String) +  " (" + pl.col("percentage").cast(pl.String) + "%)").alias("display_label")
    )

    print(counts)

def plot_distribution(df, col_name):
    col = df[col_name]

    # if col == pl.Categorical:
    #     # plot countplot

    # else:
    #     pass


In [62]:
plot_categorical_countplot(df, "Marital Status")

shape: (4, 4)
┌────────────────┬───────┬────────────┬────────────────┐
│ Marital Status ┆ count ┆ percentage ┆ display_label  │
│ ---            ┆ ---   ┆ ---        ┆ ---            │
│ cat            ┆ u32   ┆ f64        ┆ str            │
╞════════════════╪═══════╪════════════╪════════════════╡
│ married        ┆ 24928 ┆ 60.52      ┆ 24928 (60.52%) │
│ divorced       ┆ 4612  ┆ 11.2       ┆ 4612 (11.2%)   │
│ single         ┆ 11568 ┆ 28.09      ┆ 11568 (28.09%) │
│ unknown        ┆ 80    ┆ 0.19       ┆ 80 (0.19%)     │
└────────────────┴───────┴────────────┴────────────────┘


In [44]:
plot_distribution(df, "Marital Status")

Categorical



# Analysis Per Column

# Client ID

## Age

## Occupation

## Marital Status

## Credit Default

## Housing Loan

## Personal Loan

## Contact Method

## Campaign Calls

## Previous Contact Days

## Subscription Status